# **Deforum Stable Diffusion v0.7 With Automated Music Sync**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). Notebook by [deforum](https://discord.gg/upmXXsrwZc)

[Quick Guide](https://docs.google.com/document/d/1RrQv7FntzOuLg4ohjRZPVL7iptIyBhwwbcEYEW2OfcI/edit?usp=sharing) to Deforum v0.7

In [ ]:
#@markdown **NVIDIA GPU**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

# Setup

In [ ]:
#@markdown **Environment Setup**
import subprocess, time, gc, os, sys

def setup_environment():
    start_time = time.time()
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("..setting up environment")
        
        all_process = [
            ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
            ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'safetensors', 'kornia==0.6.7'],
            ['git', 'clone', 'https://github.com/deforum-art/deforum-stable-diffusion'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn','torchsde','open-clip-torch','numpngw'],
        ]
        for process in all_process:
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
        ])
        if use_xformers_for_colab:

            print("..installing xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20220701']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)

            v_card_name = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
            if 't4' in v_card_name.lower():
                name_to_download = 'T4'
            elif 'v100' in v_card_name.lower():
                name_to_download = 'V100'
            elif 'a100' in v_card_name.lower():
                name_to_download = 'A100'
            elif 'p100' in v_card_name.lower():
                name_to_download = 'P100'
            elif 'a4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/A4000'
            elif 'p5000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/P5000'
            elif 'quadro m4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/Quadro M4000'
            elif 'rtx 4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/RTX 4000'
            elif 'rtx 5000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/RTX 5000'
            else:
                print(v_card_name + ' is currently not supported with xformers flash attention in deforum!')

            if 'Non-Colab' in name_to_download:
                x_ver = 'xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl'
            else:
                x_ver = 'xformers-0.0.13.dev0-py3-none-any.whl'

            x_link = 'https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/' + name_to_download + '/' + x_ver

            all_process = [
                ['wget', '--no-verbose', '--no-clobber', x_link],
                ['pip', 'install', x_ver],
            ]

            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
    else:
        sys.path.extend([
            'src'
        ])
    end_time = time.time()
    print(f"..environment set up in {end_time-start_time:.0f} seconds")
    return

setup_environment()

import torch
import random
import clip
from IPython import display
from types import SimpleNamespace
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
from helpers.aesthetics import load_aesthetics_model

In [ ]:
#@markdown **Path Setup**

def Root():
    models_path = "models" #@param {type:"string"}
    configs_path = "configs" #@param {type:"string"}
    output_path = "outputs" #@param {type:"string"}
    mount_google_drive = False #@param {type:"boolean"}
    models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}

    #@markdown **Model Setup**
    model_config = "v1-inference.yaml" #@param ["custom","v2-inference.yaml","v2-inference-v.yaml","v1-inference.yaml"]
    model_checkpoint =  "sd-v1-4.ckpt" #@param ["custom","v2-1_768-ema-pruned.ckpt","v2-1_512-ema-pruned.ckpt","768-v-ema.ckpt","512-base-ema.ckpt","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
    custom_config_path = "" #@param {type:"string"}
    custom_checkpoint_path = "" #@param {type:"string"}
    return locals()

root = Root()
root = SimpleNamespace(**root)

root.models_path, root.output_path = get_model_output_paths(root)
root.model, root.device = load_model(root, load_on_run_all=True, check_sha256=True)

# Audio Sync settings
You can control animation settings like zoom/angle etc based on a music aplitudes. To do so, we need to analyze a music track and generate a keyframe string. Fortunatelly, there is a colab notebook that automates the whole process. You can use this [Notebook](https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/Audio_Keyframe_Generator_For_Deforum_Stable_Diffusion.ipynb) and
copy/paste the keyframe string to the 'zoom' setting below, or play with the function and generate keyframes for any other setting

# Settings

In [ ]:
def DeforumAnimArgs():

    #@markdown ####**Animation:**
    animation_mode = '2D' #@param ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    max_frames = 719 #@param {type:"number"}
    border = 'replicate' #@param ['wrap', 'replicate'] {type:'string'}

    #@markdown ####**Motion Parameters:**
    angle = "0: (-1), 50: (0), 100: (1), 150: (1), 200: (0), 250: (-1), 300: (1), 350: (0), 400: (1), 450: (-1), 500: (1), 550: (0), 600: (0), 650: (-1), 700: (1), "#@param {type:"string"}
    zoom = "0: (1.03), 1: (1.03), 2: (1.03), 3: (1.03), 4: (1.03), 5: (1.03), 6: (1.03), 7: (1.03), 8: (1.03), 9: (1.03), 10: (1.03), 11: (1.03), 12: (1.03), 13: (1.03), 14: (1.04), 15: (1.03), 16: (1.03), 17: (1.03), 18: (1.03), 19: (1.03), 20: (1.03), 21: (1.03), 22: (1.03), 23: (1.03), 24: (1.03), 25: (1.03), 26: (1.03), 27: (1.03), 28: (1.03), 29: (1.03), 30: (1.03), 31: (1.45), 32: (1.03), 33: (1.03), 34: (1.03), 35: (1.03), 36: (1.03), 37: (1.03), 38: (1.03), 39: (1.03), 40: (1.03), 41: (1.03), 42: (1.03), 43: (1.03), 44: (1.03), 45: (1.03), 46: (1.03), 47: (1.07), 48: (1.08), 49: (1.03), 50: (1.03), 51: (1.03), 52: (1.03), 53: (1.03), 54: (1.03), 55: (1.03), 56: (1.03), 57: (1.03), 58: (1.09), 59: (1.29), 60: (1.03), 61: (1.03), 62: (1.03), 63: (1.03), 64: (1.03), 65: (1.03), 66: (1.03), 67: (1.03), 68: (1.03), 69: (1.03), 70: (1.03), 71: (1.03), 72: (1.03), 73: (1.03), 74: (1.03), 75: (1.28), 76: (1.03), 77: (1.03), 78: (1.03), 79: (1.03), 80: (1.03), 81: (1.03), 82: (1.03), 83: (1.03), 84: (1.03), 85: (1.03), 86: (1.03), 87: (1.03), 88: (1.03), 89: (1.03), 90: (1.03), 91: (1.03), 92: (1.34), 93: (1.03), 94: (1.03), 95: (1.03), 96: (1.03), 97: (1.03), 98: (1.03), 99: (1.03), 100: (1.03), 101: (1.03), 102: (1.03), 103: (1.40), 104: (1.03), 105: (1.03), 106: (1.03), 107: (1.03), 108: (1.03), 109: (1.03), 110: (1.03), 111: (1.03), 112: (1.03), 113: (1.03), 114: (1.03), 115: (1.03), 116: (1.03), 117: (1.03), 118: (1.03), 119: (1.11), 120: (1.19), 121: (1.03), 122: (1.03), 123: (1.03), 124: (1.03), 125: (1.03), 126: (1.03), 127: (1.03), 128: (1.03), 129: (1.03), 130: (1.03), 131: (1.03), 132: (1.03), 133: (1.03), 134: (1.03), 135: (1.03), 136: (1.29), 137: (1.03), 138: (1.03), 139: (1.03), 140: (1.03), 141: (1.03), 142: (1.03), 143: (1.03), 144: (1.03), 145: (1.03), 146: (1.03), 147: (1.36), 148: (1.03), 149: (1.03), 150: (1.03), 151: (1.03), 152: (1.03), 153: (1.03), 154: (1.03), 155: (1.03), 156: (1.03), 157: (1.03), 158: (1.03), 159: (1.03), 160: (1.03), 161: (1.03), 162: (1.03), 163: (1.03), 164: (1.03), 165: (1.03), 166: (1.03), 167: (1.03), 168: (1.03), 169: (1.03), 170: (1.03), 171: (1.03), 172: (1.03), 173: (1.03), 174: (1.03), 175: (1.03), 176: (1.03), 177: (1.03), 178: (1.03), 179: (1.03), 180: (1.10), 181: (1.12), 182: (1.03), 183: (1.03), 184: (1.03), 185: (1.03), 186: (1.03), 187: (1.03), 188: (1.03), 189: (1.03), 190: (1.03), 191: (1.05), 192: (1.21), 193: (1.03), 194: (1.03), 195: (1.03), 196: (1.03), 197: (1.03), 198: (1.03), 199: (1.03), 200: (1.03), 201: (1.03), 202: (1.03), 203: (1.03), 204: (1.03), 205: (1.03), 206: (1.03), 207: (1.03), 208: (1.29), 209: (1.03), 210: (1.03), 211: (1.03), 212: (1.03), 213: (1.03), 214: (1.03), 215: (1.03), 216: (1.03), 217: (1.03), 218: (1.03), 219: (1.03), 220: (1.03), 221: (1.03), 222: (1.03), 223: (1.03), 224: (1.03), 225: (1.16), 226: (1.03), 227: (1.03), 228: (1.03), 229: (1.03), 230: (1.03), 231: (1.03), 232: (1.03), 233: (1.03), 234: (1.03), 235: (1.03), 236: (1.67), 237: (1.03), 238: (1.03), 239: (1.03), 240: (1.03), 241: (1.03), 242: (1.03), 243: (1.03), 244: (1.03), 245: (1.03), 246: (1.03), 247: (1.03), 248: (1.03), 249: (1.03), 250: (1.03), 251: (1.03), 252: (1.06), 253: (1.14), 254: (1.03), 255: (1.03), 256: (1.03), 257: (1.03), 258: (1.03), 259: (1.03), 260: (1.03), 261: (1.03), 262: (1.03), 263: (1.03), 264: (1.03), 265: (1.03), 266: (1.03), 267: (1.03), 268: (1.03), 269: (1.32), 270: (1.04), 271: (1.03), 272: (1.03), 273: (1.03), 274: (1.03), 275: (1.03), 276: (1.03), 277: (1.03), 278: (1.03), 279: (1.03), 280: (1.33), 281: (1.04), 282: (1.03), 283: (1.03), 284: (1.03), 285: (1.03), 286: (1.03), 287: (1.03), 288: (1.03), 289: (1.03), 290: (1.03), 291: (1.03), 292: (1.03), 293: (1.03), 294: (1.03), 295: (1.03), 296: (1.03), 297: (1.36), 298: (1.03), 299: (1.03), 300: (1.03), 301: (1.03), 302: (1.03), 303: (1.03), 304: (1.03), 305: (1.03), 306: (1.03), 307: (1.03), 308: (1.03), 309: (1.03), 310: (1.03), 311: (1.03), 312: (1.03), 313: (1.06), 314: (1.07), 315: (1.03), 316: (1.03), 317: (1.03), 318: (1.03), 319: (1.03), 320: (1.03), 321: (1.03), 322: (1.03), 323: (1.03), 324: (1.11), 325: (1.14), 326: (1.03), 327: (1.03), 328: (1.03), 329: (1.03), 330: (1.03), 331: (1.03), 332: (1.03), 333: (1.03), 334: (1.03), 335: (1.03), 336: (1.03), 337: (1.03), 338: (1.03), 339: (1.03), 340: (1.03), 341: (1.03), 342: (1.03), 343: (1.03), 344: (1.03), 345: (1.03), 346: (1.03), 347: (1.03), 348: (1.03), 349: (1.03), 350: (1.03), 351: (1.03), 352: (1.03), 353: (1.03), 354: (1.03), 355: (1.03), 356: (1.03), 357: (1.03), 358: (1.20), 359: (1.03), 360: (1.03), 361: (1.03), 362: (1.03), 363: (1.03), 364: (1.03), 365: (1.03), 366: (1.03), 367: (1.03), 368: (1.03), 369: (1.27), 370: (1.03), 371: (1.03), 372: (1.03), 373: (1.03), 374: (1.03), 375: (1.03), 376: (1.03), 377: (1.03), 378: (1.03), 379: (1.03), 380: (1.03), 381: (1.03), 382: (1.03), 383: (1.03), 384: (1.03), 385: (1.04), 386: (1.05), 387: (1.03), 388: (1.03), 389: (1.03), 390: (1.03), 391: (1.03), 392: (1.03), 393: (1.03), 394: (1.03), 395: (1.03), 396: (1.03), 397: (1.03), 398: (1.03), 399: (1.03), 400: (1.03), 401: (1.03), 402: (1.15), 403: (1.03), 404: (1.03), 405: (1.03), 406: (1.03), 407: (1.03), 408: (1.03), 409: (1.03), 410: (1.03), 411: (1.03), 412: (1.03), 413: (1.22), 414: (1.05), 415: (1.03), 416: (1.03), 417: (1.03), 418: (1.03), 419: (1.03), 420: (1.03), 421: (1.03), 422: (1.03), 423: (1.03), 424: (1.03), 425: (1.03), 426: (1.03), 427: (1.03), 428: (1.03), 429: (1.03), 430: (1.72), 431: (1.03), 432: (1.03), 433: (1.03), 434: (1.03), 435: (1.03), 436: (1.03), 437: (1.03), 438: (1.03), 439: (1.03), 440: (1.03), 441: (1.03), 442: (1.03), 443: (1.03), 444: (1.03), 445: (1.03), 446: (1.05), 447: (1.09), 448: (1.03), 449: (1.03), 450: (1.03), 451: (1.03), 452: (1.03), 453: (1.03), 454: (1.03), 455: (1.03), 456: (1.03), 457: (1.04), 458: (1.19), 459: (1.03), 460: (1.03), 461: (1.03), 462: (1.03), 463: (1.03), 464: (1.03), 465: (1.03), 466: (1.03), 467: (1.03), 468: (1.03), 469: (1.03), 470: (1.03), 471: (1.03), 472: (1.03), 473: (1.03), 474: (1.22), 475: (1.04), 476: (1.03), 477: (1.03), 478: (1.03), 479: (1.03), 480: (1.03), 481: (1.03), 482: (1.03), 483: (1.03), 484: (1.03), 485: (1.03), 486: (1.03), 487: (1.03), 488: (1.03), 489: (1.03), 490: (1.03), 491: (1.22), 492: (1.03), 493: (1.03), 494: (1.03), 495: (1.03), 496: (1.03), 497: (1.03), 498: (1.03), 499: (1.03), 500: (1.03), 501: (1.03), 502: (1.44), 503: (1.03), 504: (1.03), 505: (1.03), 506: (1.03), 507: (1.03), 508: (1.03), 509: (1.03), 510: (1.03), 511: (1.03), 512: (1.03), 513: (1.03), 514: (1.03), 515: (1.03), 516: (1.03), 517: (1.03), 518: (1.03), 519: (1.03), 520: (1.03), 521: (1.03), 522: (1.03), 523: (1.03), 524: (1.03), 525: (1.03), 526: (1.03), 527: (1.03), 528: (1.03), 529: (1.03), 530: (1.03), 531: (1.03), 532: (1.03), 533: (1.03), 534: (1.03), 535: (1.15), 536: (1.05), 537: (1.03), 538: (1.03), 539: (1.03), 540: (1.03), 541: (1.03), 542: (1.03), 543: (1.03), 544: (1.03), 545: (1.03), 546: (1.15), 547: (1.05), 548: (1.03), 549: (1.03), 550: (1.03), 551: (1.03), 552: (1.03), 553: (1.03), 554: (1.03), 555: (1.03), 556: (1.03), 557: (1.03), 558: (1.03), 559: (1.03), 560: (1.03), 561: (1.03), 562: (1.03), 563: (2.03), 564: (1.03), 565: (1.03), 566: (1.03), 567: (1.03), 568: (1.03), 569: (1.03), 570: (1.03), 571: (1.03), 572: (1.03), 573: (1.03), 574: (1.03), 575: (1.03), 576: (1.03), 577: (1.03), 578: (1.03), 579: (1.03), 580: (1.06), 581: (1.03), 582: (1.03), 583: (1.03), 584: (1.03), 585: (1.03), 586: (1.03), 587: (1.03), 588: (1.03), 589: (1.03), 590: (1.04), 591: (1.31), 592: (1.03), 593: (1.03), 594: (1.03), 595: (1.03), 596: (1.03), 597: (1.03), 598: (1.03), 599: (1.03), 600: (1.03), 601: (1.03), 602: (1.03), 603: (1.03), 604: (1.03), 605: (1.03), 606: (1.03), 607: (1.37), 608: (1.05), 609: (1.03), 610: (1.03), 611: (1.03), 612: (1.03), 613: (1.03), 614: (1.03), 615: (1.03), 616: (1.03), 617: (1.03), 618: (1.03), 619: (1.03), 620: (1.03), 621: (1.03), 622: (1.03), 623: (1.03), 624: (1.26), 625: (1.03), 626: (1.03), 627: (1.03), 628: (1.03), 629: (1.03), 630: (1.03), 631: (1.03), 632: (1.03), 633: (1.03), 634: (1.03), 635: (1.38), 636: (1.03), 637: (1.03), 638: (1.03), 639: (1.03), 640: (1.03), 641: (1.03), 642: (1.03), 643: (1.03), 644: (1.03), 645: (1.03), 646: (1.03), 647: (1.03), 648: (1.03), 649: (1.03), 650: (1.03), 651: (1.03), 652: (1.12), 653: (1.03), 654: (1.03), 655: (1.03), 656: (1.03), 657: (1.03), 658: (1.03), 659: (1.03), 660: (1.03), 661: (1.03), 662: (1.03), 663: (1.03), 664: (1.03), 665: (1.03), 666: (1.03), 667: (1.03), 668: (1.03), 669: (1.03), 670: (1.03), 671: (1.03), 672: (1.03), 673: (1.03), 674: (1.03), 675: (1.03), 676: (1.03), 677: (1.03), 678: (1.03), 679: (1.03), 680: (1.03), 681: (1.03), 682: (1.03), 683: (1.03), 684: (1.03), 685: (1.03), 686: (1.03), 687: (1.03), 688: (1.03), 689: (1.03), 690: (1.03), 691: (1.03), 692: (1.03), 693: (1.03), 694: (1.03), 695: (1.03), 696: (1.03), 697: (1.03), 698: (1.03), 699: (1.03), 700: (1.03), 701: (1.03), 702: (1.03), 703: (1.03), 704: (1.03), 705: (1.03), 706: (1.03), 707: (1.03), 708: (1.03), 709: (1.03), 710: (1.03), 711: (1.03), 712: (1.03), 713: (1.03), 714: (1.03), 715: (1.03), 716: (1.03), 717: (1.03), 718: (1.03), 719: (1.03), "#@param {type:"string"}
    translation_x = "0:(10*sin(2*3.14*t/10))"#@param {type:"string"}
    translation_y = "0:(0)"#@param {type:"string"}
    translation_z = "0:(10)"#@param {type:"string"}
    rotation_3d_x = "0:(0)"#@param {type:"string"}
    rotation_3d_y = "0:(0)"#@param {type:"string"}
    rotation_3d_z = "0:(0)"#@param {type:"string"}
    flip_2d_perspective = False #@param {type:"boolean"}
    perspective_flip_theta = "0:(0)"#@param {type:"string"}
    perspective_flip_phi = "0:(t%15)"#@param {type:"string"}
    perspective_flip_gamma = "0:(0)"#@param {type:"string"}
    perspective_flip_fv = "0:(53)"#@param {type:"string"}
    noise_schedule = "0: (0.02)"#@param {type:"string"}
    strength_schedule = "0: (0.60)"#@param {type:"string"}
    contrast_schedule = "0: (1.0)"#@param {type:"string"}
    hybrid_video_comp_alpha_schedule = "0:(1)" #@param {type:"string"}
    hybrid_video_comp_mask_blend_alpha_schedule = "0:(0.5)" #@param {type:"string"}
    hybrid_video_comp_mask_contrast_schedule = "0:(1)" #@param {type:"string"}
    hybrid_video_comp_mask_auto_contrast_cutoff_high_schedule =  "0:(100)" #@param {type:"string"}
    hybrid_video_comp_mask_auto_contrast_cutoff_low_schedule =  "0:(0)" #@param {type:"string"}

    #@markdown ####**Unsharp mask (anti-blur) Parameters:**
    kernel_schedule = "0: (5)"#@param {type:"string"}
    sigma_schedule = "0: (1.0)"#@param {type:"string"}
    amount_schedule = "0: (0.2)"#@param {type:"string"}
    threshold_schedule = "0: (0.0)"#@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'Match Frame 0 LAB' #@param ['None', 'Match Frame 0 HSV', 'Match Frame 0 LAB', 'Match Frame 0 RGB', 'Video Input'] {type:'string'}
    color_coherence_video_every_N_frames = 1 #@param {type:"integer"}
    diffusion_cadence = '2' #@param ['1','2','3','4','5','6','7','8'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3#@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 40#@param {type:"number"}
    padding_mode = 'border'#@param ['border', 'reflection', 'zeros'] {type:'string'}
    sampling_mode = 'bicubic'#@param ['bicubic', 'bilinear', 'nearest'] {type:'string'}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4'#@param {type:"string"}
    extract_nth_frame = 1#@param {type:"number"}
    overwrite_extracted_frames = True #@param {type:"boolean"}
    use_mask_video = False #@param {type:"boolean"}
    video_mask_path ='/content/video_in.mp4'#@param {type:"string"}

    #@markdown ####**Hybrid Video for 2D/3D Animation Mode:**
    hybrid_video_generate_inputframes = False #@param {type:"boolean"}
    hybrid_video_use_first_frame_as_init_image = True #@param {type:"boolean"}
    hybrid_video_motion = "None" #@param ['None','Optical Flow','Perspective','Affine']
    hybrid_video_flow_method = "Farneback" #@param ['Farneback','DenseRLOF','SF']
    hybrid_video_composite = False #@param {type:"boolean"}
    hybrid_video_comp_mask_type = "None" #@param ['None', 'Depth', 'Video Depth', 'Blend', 'Difference']
    hybrid_video_comp_mask_inverse = False #@param {type:"boolean"}
    hybrid_video_comp_mask_equalize = "None" #@param  ['None','Before','After','Both']
    hybrid_video_comp_mask_auto_contrast = False #@param {type:"boolean"}
    hybrid_video_comp_save_extra_frames = False #@param {type:"boolean"}
    hybrid_video_use_video_as_mse_image = False #@param {type:"boolean"}

    #@markdown ####**Interpolation:**
    interpolate_key_frames = False #@param {type:"boolean"}
    interpolate_x_frames = 4 #@param {type:"number"}
    
    #@markdown ####**Resume Animation:**
    resume_from_timestring = False #@param {type:"boolean"}
    resume_timestring = "20220829210106" #@param {type:"string"}

    return locals()

In [ ]:
prompts = [
    "a beautiful lake by Asher Brown Durand, trending on Artstation", # the first prompt I want
    "a beautiful portrait of a woman by Artgerm, trending on Artstation", # the second prompt I want
    #"this prompt I don't want it I commented it out",
    #"a nousr robot, trending on Artstation", # use "nousr robot" with the robot diffusion model (see model_checkpoint setting)
    #"touhou 1girl komeiji_koishi portrait, green hair", # waifu diffusion prompts can use danbooru tag groups (see model_checkpoint)
    #"this prompt has weights if prompt weighting enabled:2 can also do negative:-2", # (see prompt_weighting)
]

animation_prompts = {
    0: "a grayscale painting beautiful lonely girl waving goodbye, by Mandy Jurgens, trending on artstation",
    200: "a grayscale painting of void post mortem, digital art, by Alayna Danner, hd, trending on artstation",
    400: "a grayscale painting of a gray prison, hd trending on artstation",
    600: "a grayscale painting of the feeling of loneliness, hd, trending on Artstation",
}

In [ ]:
#@markdown **Load Settings**
override_settings_with_file = False #@param {type:"boolean"}
settings_file = "custom" #@param ["custom", "512x512_aesthetic_0.json","512x512_aesthetic_1.json","512x512_colormatch_0.json","512x512_colormatch_1.json","512x512_colormatch_2.json","512x512_colormatch_3.json"]
custom_settings_file = "/content/drive/MyDrive/Settings.txt"#@param {type:"string"}

def DeforumArgs():
    #@markdown **Image Settings**
    W = 720 #@param
    H = 405 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    bit_depth_output = 8 #@param [8, 16, 32] {type:"raw"}

    #@markdown **Sampling Settings**
    seed = -1 #@param
    sampler = 'dpmpp_2s_a' #@param ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim", "dpm_fast", "dpm_adaptive", "dpmpp_2s_a", "dpmpp_2m"]
    steps = 50 #@param
    scale = 8 #@param
    ddim_eta = 0.0 #@param
    dynamic_threshold = None
    static_threshold = None   

    #@markdown **Save & Display Settings**
    save_samples = True #@param {type:"boolean"}
    save_settings = True #@param {type:"boolean"}
    display_samples = True #@param {type:"boolean"}
    save_sample_per_step = False #@param {type:"boolean"}
    show_sample_per_step = False #@param {type:"boolean"}

    #@markdown **Prompt Settings**
    prompt_weighting = True #@param {type:"boolean"}
    normalize_prompt_weights = True #@param {type:"boolean"}
    log_weighted_subprompts = False #@param {type:"boolean"}

    #@markdown **Batch Settings**
    n_batch = 1 #@param
    batch_name = "StableFun" #@param {type:"string"}
    filename_format = "{timestring}_{index}_{prompt}.png" #@param ["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png"]
    seed_behavior = "iter" #@param ["iter","fixed","random","ladder","alternate"]
    seed_iter_N = 1 #@param {type:'integer'}
    make_grid = False #@param {type:"boolean"}
    grid_rows = 2 #@param 
    outdir = get_output_folder(root.output_path, batch_name)

    #@markdown **Init Settings**
    use_init = False #@param {type:"boolean"}
    strength = 0.1 #@param {type:"number"}
    strength_0_no_init = True # Set the strength to 0 automatically when no init image is used
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    # Whiter areas of the mask are areas that change more
    use_mask = False #@param {type:"boolean"}
    use_alpha_as_mask = False # use the alpha channel of the init image as the mask
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" #@param {type:"string"}
    invert_mask = False #@param {type:"boolean"}
    # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
    mask_brightness_adjust = 1.0  #@param {type:"number"}
    mask_contrast_adjust = 1.0  #@param {type:"number"}
    # Overlay the masked image at the end of the generation so it does not get degraded by encoding and decoding
    overlay_mask = True  # {type:"boolean"}
    # Blur edges of final overlay mask, if used. Minimum = 0 (no blur)
    mask_overlay_blur = 5 # {type:"number"}

    #@markdown **Exposure/Contrast Conditional Settings**
    mean_scale = 0 #@param {type:"number"}
    var_scale = 0 #@param {type:"number"}
    exposure_scale = 0 #@param {type:"number"}
    exposure_target = 0.5 #@param {type:"number"}

    #@markdown **Color Match Conditional Settings**
    colormatch_scale = 0 #@param {type:"number"}
    colormatch_image = "https://www.saasdesign.io/wp-content/uploads/2021/02/palette-3-min-980x588.png" #@param {type:"string"}
    colormatch_n_colors = 4 #@param {type:"number"}
    ignore_sat_weight = 0 #@param {type:"number"}

    #@markdown **CLIP\Aesthetics Conditional Settings**
    clip_name = 'ViT-L/14' #@param ['ViT-L/14', 'ViT-L/14@336px', 'ViT-B/16', 'ViT-B/32']
    clip_scale = 0 #@param {type:"number"}
    aesthetics_scale = 0 #@param {type:"number"}
    cutn = 1 #@param {type:"number"}
    cut_pow = 0.0001 #@param {type:"number"}

    #@markdown **Other Conditional Settings**
    init_mse_scale = 0 #@param {type:"number"}
    init_mse_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}

    blue_scale = 0 #@param {type:"number"}
    
    #@markdown **Conditional Gradient Settings**
    gradient_wrt = 'x0_pred' #@param ["x", "x0_pred"]
    gradient_add_to = 'both' #@param ["cond", "uncond", "both"]
    decode_method = 'linear' #@param ["autoencoder","linear"]
    grad_threshold_type = 'dynamic' #@param ["dynamic", "static", "mean", "schedule"]
    clamp_grad_threshold = 0.2 #@param {type:"number"}
    clamp_start = 0.2 #@param
    clamp_stop = 0.01 #@param
    grad_inject_timing = list(range(1,10)) #@param

    #@markdown **Speed vs VRAM Settings**
    cond_uncond_sync = True #@param {type:"boolean"}

    n_samples = 1 # doesnt do anything
    precision = 'autocast' 
    C = 4
    f = 8

    prompt = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None
    seed_internal = 0

    return locals()

args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    load_args(args_dict, anim_args_dict, settings_file, custom_settings_file, verbose=False)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

# Load clip model if using clip guidance
if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
    root.clip_model = clip.load(args.clip_name, jit=False)[0].eval().requires_grad_(False).to(root.device)
    if (args.aesthetics_scale > 0):
        root.aesthetics_model = load_aesthetics_model(args, root)

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

# dispatch to appropriate renderer
if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(args, anim_args, animation_prompts, root)
else:
    render_image_batch(args, prompts, root)

# Create Video From Frames

In [ ]:
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 24 #@param {type:"number"}
#@markdown **Manual Settings**
use_manual_settings = False #@param {type:"boolean"}
image_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/StableFun/20230101212135_%05d.png" #@param {type:"string"}
mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/StableFun/output_video.mp4" #@param {type:"string"}
render_steps = False  #@param {type: 'boolean'}
path_name_modifier = "x0_pred" #@param ["x0_pred","x"]
make_gif = False
bitdepth_extension = "exr" if args.bit_depth_output == 32 else "png"

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    import os
    import subprocess
    from base64 import b64encode

    print(f"{image_path} -> {mp4_path}")

    if use_manual_settings:
        max_frames = "719" #@param {type:"string"}
    else:
        if render_steps: # render steps from a single image
            fname = f"{path_name_modifier}_%05d.png"
            all_step_dirs = [os.path.join(args.outdir, d) for d in os.listdir(args.outdir) if os.path.isdir(os.path.join(args.outdir,d))]
            newest_dir = max(all_step_dirs, key=os.path.getmtime)
            image_path = os.path.join(newest_dir, fname)
            print(f"Reading images from {image_path}")
            mp4_path = os.path.join(newest_dir, f"{args.timestring}_{path_name_modifier}.mp4")
            max_frames = str(args.steps)
        else: # render images for a video
            image_path = os.path.join(args.outdir, f"{args.timestring}_%05d.{bitdepth_extension}")
            mp4_path = os.path.join(args.outdir, f"{args.timestring}.mp4")
            max_frames = str(anim_args.max_frames)

    # make video
    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', bitdepth_extension,
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-frames:v', max_frames,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        '-pattern_type', 'sequence',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display(display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
    
    if make_gif:
         gif_path = os.path.splitext(mp4_path)[0]+'.gif'
         cmd_gif = [
             'ffmpeg',
             '-y',
             '-i', mp4_path,
             '-r', str(fps),
             gif_path
         ]
         process_gif = subprocess.Popen(cmd_gif, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
skip_disconnect_for_run_all = True #@param {type: 'boolean'}

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()